### Import data to MongoDB

In [2]:
# %pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 KB 36.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["ratingRager"]
collection = db["restaurant_details"]

with open("data/restaurant_reviews_final.json", "r") as file:
    data = json.load(file)


document = collection.insert_many(data.values())

for document in collection.find():
    print(document)

### Import Data to SQLite3

In [22]:
import json

with open("../restaurant_reviews_final.json", "r") as file:
    data = json.load(file)

In [23]:
import sqlite3
from datetime import datetime

conn = sqlite3.connect('../ratingRager.db')
cursor = conn.cursor()

In [24]:
# Create tables (if they don't exist)
cursor.execute('''
CREATE TABLE IF NOT EXISTS restaurants (
    id INTEGER PRIMARY KEY,
    name TEXT,
    rating REAL,
    rating_volume INTEGER,
    address TEXT,
    latitude REAL,
    longitude REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY,
    restaurant_id INTEGER,
    author_name TEXT,
    rating INTEGER,
    description TEXT,
    date DATETIME
)
''')

In [26]:
for key, value in data.items():
    if "rating" not in value:
        print("Restaurant Closed or lack information: ", value["name"], value["formatted_address"])
        continue

    restaurant_name = value["name"]
    restaurant_rating = value["rating"]
    rating_volume = value["user_ratings_total"]
    address = value["formatted_address"]
    latitude = value["geometry"]["location"]["lat"]
    longitude = value["geometry"]["location"]["lng"]

    cursor.execute('''
    INSERT INTO restaurants (name, rating, rating_volume, address, latitude, longitude)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (restaurant_name, restaurant_rating, rating_volume, address, latitude, longitude))

    # Get the restaurant's ID (last inserted row)
    restaurant_id = cursor.lastrowid

    if 'reviews' not in value:
        continue

    reviewer_name = []
    for review in value["reviews"]:
        if review["author_name"] in reviewer_name:
            # Skip duplicate reviewer
            continue

        reviewer_name.append(review)
        author_name = review["author_name"]
        rating = review["rating"]
        description = review["text"]
        # Convert epoch time to datetime
        review_time = datetime.fromtimestamp(review["time"]).strftime('%m-%d-%Y %H:%M:%S')

        cursor.execute('''
        INSERT INTO reviews (restaurant_id, author_name, rating, description, date)
        VALUES (?, ?, ?, ?, ?)
        ''', (restaurant_id, author_name, rating, description, review_time))

conn.commit()
conn.close()

Restaurant Closed or lack information:  Pizzaholic Inc. 144 Rundlehill Dr NE, Calgary, AB T1Y 2P6, Canada
Restaurant Closed or lack information:  Jasmine cake 193 Saddlebrook Way NE, Calgary, AB T3J 0B5, Canada
Restaurant Closed or lack information:  Bento Sushi 505 Quarry Park Blvd SE, Calgary, AB T2C 5M1, Canada
Restaurant Closed or lack information:  Wakhra Swad East Indian Restaurant & Bar 3870 Cornerstone Blvd NE Unit 310-320, Calgary, AB T3N 2E8, Canada
Restaurant Closed or lack information:  Rangoli 6145 6 St SE, Calgary, AB T2H 1L9, Canada
Restaurant Closed or lack information:  Wings Avenue 2717 Sunridge Way NE, Calgary, AB T1Y 7K7, Canada
Restaurant Closed or lack information:  Boreal Cuisine 4216 64 Ave SE #15, Calgary, AB T2C 2B3, Canada
Restaurant Closed or lack information:  J P Burgers Ltd 855 2 St SW, Calgary, AB T2P 4J7, Canada
Restaurant Closed or lack information:  Bento Sushi 917 85 St SW #100, Calgary, AB T3H 4C8, Canada
Restaurant Closed or lack information:  Bent